In [1]:
# libs
import pandas as pd
import sqlite3
import warnings
import os
import logging
warnings.filterwarnings('ignore')
pd.option_context(10,5)
from unidecode import unidecode

In [4]:

#define o caminho do diretório atual
current_dir = os.getcwd()
file_logs = current_dir.replace(r'vr\dados',r'logs\vr.log')
# configurando o registro de logs
logging.basicConfig(level=logging.DEBUG, filename=file_logs,encoding='utf-8', format="%(asctime)s - %(levelname)s - %(message)s")

# pega o arquivo gerado
base_vr = current_dir + r'\BASE_VR.csv'
base_vr

'c:\\Users\\ABRASEL NACIONAL\\Documents\\GitHub\\ESTABELECIMENTOS_ETL\\vr\\dados\\BASE_VR.csv'

In [6]:
# carregada os dados no dataframe pandas aqui, simples né?

dados  = pd.read_csv(base_vr, sep=';', dtype='string')
dados

,CNPJ,ESTABELECIMENTOS,ENDERECO,BAIRRO,CIDADE,UF,CEP,TELEFONE,EMAIL,LATITUDE,LONGITUDE,BANDEIRA
0,04979388000175,JOTA COMERCIAL,"PRES VARGAS,0",SAO FRANCISCO,TERRA SANTA,PA,68285-000,99914757,<NA>,-2.1061219,-56.4867736,VR
1,33032113000128,MERCANTIL J&S,"1 TV 3 DE OUTUBRO,391",JUVENIL,TERRA SANTA,PA,68285-000,991525759,<NA>,-2.1069839,-56.4967818,VR
2,83906149000161,COMERCIAL GENTIL,"CANTIDIANO DE MENDONCA,269",CENTRO,TOME-ACU,PA,68680-000,91715220,<NA>,-2.4186074,-48.1497998,VR
3,39344257000104,CASA DE CARNES BOI NA BRASA,"BRASIL,4331",CENTRO,ALTA FLORESTA D'OESTE,RO,76954-000,993987901,<NA>,-11.9289039,-61.9907462,VR
4,01920923000124,COMERCIAL VENTURA,"PIAUI,5371",PRINCESA IZABEL,ALTA FLORESTA D'OESTE,RO,78994-000,36413031,<NA>,-11.9245228,-62.0026549,VR
...,...,...,...,...,...,...,...,...,...,...,...,...
54913,08925650000150,REST. DOURADO,"EQNL 21/23 BLOCO 'B' LOJA,06",TAGUATINGA,BRASILIA,DF,72120-190,12345678,<NA>,-15.8141117,-48.0908074,VR
54914,27588550000120,BIROSCA DO LEO,"QR 120 CONJUNTO 1,1",SAMAMBAIA SUL (SAMAMBAIA),BRASILIA,DF,72304-001,85898974,<NA>,-15.8672316,-48.0633122,VR
54915,33724144000140,SIVANILDO FERNANDES SIMPLICIO,"QNL 23 BLOCO E,14",TAGUATINGA N (TAGUAT,BRASILIA,DF,72152-315,94038092,<NA>,-15.8125112,-48.0919444,VR
54916,33507503000107,HERBALIFE,"EQNL 5/7 BLOCO B,202",TAGUATINGA NORTE (TAGUATINGA),BRASILIA,DF,72150-512,83617690,<NA>,-15.8231662,-48.0816325,VR


In [7]:
dados['CIDADE'] = [unidecode(i) for i in dados['CIDADE'].map(str)]
dados['CIDADE'] = [i.replace('-', ' ') for i in dados['CIDADE'].map(str)]

In [ ]:
# Salva tudo novamente desta vez com um csv e no banco de dados, a galera gosta de "variedades"
dados.to_csv(base_vr,sep=';', index=False, encoding='utf-8')
#Criar uma conexão com o banco de dados sqlite
db_file = current_dir.replace('vr\dados', r'app\files\database.db')
conn = sqlite3.connect(database=db_file)

#Converter o dataframe em uma tabela no banco de dados
"""
O parâmetro if_exists=`append` verifica se a tabela já existe e incrementa os dados
O parâmetro index=False evita que o índice do dataframe seja inserido na tabela
O parâmetro dtype define o tipo de cada coluna na tabela
"""
dados.to_sql('tb_vr', conn, 
             if_exists='replace', index=False, 
             dtype={'CNPJ': 'TEXT PRIMARY KEY', 
                    'ESTABELECIMENTOS': 'TEXT', 'ENDERECO': 'TEXT', 
                    'BAIRRO': 'TEXT', 'CIDADE': 'TEXT', 'UF': 'TEXT', 
                    'CEP': 'TEXT', 'TELEFONE': 'TEXT', 
                    'EMAIL': 'TEXT', 'LATITUDE': 'TEXT', 
                    'LONGITUDE': 'TEXT', 'BANDEIRA': 'TEXT'})
# Finaliza a transação
conn.commit()
# Executa o comando VACUUM para compactar o banco de dados
conn.execute('VACUUM')

# Fechar a conexão com o banco de dados
conn.close()